In [ ]:
# Breast Cancer EDA & Baseline Model Training

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [ ]:
# Optional: MLflow logging (can be commented out)
import mlflow

In [ ]:
# Load dataset
df = pd.read_csv("../data/breast_cancer.csv")

In [ ]:
# Show basic info
print("📊 Dataset shape:", df.shape)
print(df.head())

In [ ]:
# Replace target: M → 1 (Malignant), B → 0 (Benign)
df["diagnosis"] = df["diagnosis"].map({"M": 1, "B": 0})

In [ ]:
# Check class distribution
sns.countplot(x="diagnosis", data=df)
plt.title("Distribution of Diagnosis")
plt.xlabel("Diagnosis (1 = Malignant, 0 = Benign)")
plt.ylabel("Count")
plt.show()

In [ ]:
# Correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(numeric_only=True), cmap="coolwarm", annot=False)
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
# Features & target
X = df.drop(columns=["id", "diagnosis", "Unnamed: 32"], errors="ignore")
y = df["diagnosis"]

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [ ]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Train Logistic Regression
log_model = LogisticRegression(max_iter=1000, random_state=42)
log_model.fit(X_train_scaled, y_train)
y_pred_log = log_model.predict(X_test_scaled)
print("📈 Logistic Regression")
print(classification_report(y_test, y_pred_log))
print("ROC AUC:", roc_auc_score(y_test, log_model.predict_proba(X_test_scaled)[:, 1]))

In [ ]:
# Train Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("\n🌲 Random Forest")
print(classification_report(y_test, y_pred_rf))
print("ROC AUC:", roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1]))

In [ ]:
# MLflow Logging
with mlflow.start_run(run_name="EDA_LogReg_vs_RF"):
    mlflow.log_metric("roc_auc_logreg", roc_auc_score(y_test, log_model.predict_proba(X_test_scaled)[:, 1]))
    mlflow.log_metric("roc_auc_rf", roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1]))
    mlflow.log_param("logreg_max_iter", 1000)
    mlflow.sklearn.log_model(rf_model, "random_forest_model")
    mlflow.set_tag("notebook", "eda_model_training.ipynb")

print("\n✅ Done.")